To setup the environment run the following commands on kaggle

In [1]:
!cp /kaggle/input/a3-1-1/A3/environment.yml /kaggle/working/
!cp /kaggle/input/a3-1-1/A3/install.sh /kaggle/working/

!bash install.sh

Installing dependencies from environment.yml using pip...
Environment setup completed successfully!


Run your .py file on CLI using the following command

```bash
!python bird.py path_to_dataset train bird.pth
!python bird.py /kaggle/input/identify-the-birds/Birds/train test /kaggle/working/bird.pth
```

# Models to be tested

### Preprocessing

1. Load Images as tensors ✅ 
2. Transform the images  
    - Padding ❌
    - Cropping ❌
    - Resize-ing ✅ 
3. Scale the images ✅ 
4. Normalize the images ✅ 

### Model Training

- Normal Architecture with EffNet scaling laws ✅
  > Compound Scaling Method ✅
- Add BatchNorm ✅
- Scale the network ✅
- Add Skip Connections ✅
- Scale the network ✅
- Try ResNet Like Architecture ✅
- Try GoogleNet like Architecture ❌
- Try Removing Classimbalance by
  1. Data Augmentation + OverSampling 
  2. Weighted Classification ❌
  3. Weighted Sampler ❌
  3. Focal Loss ❌
- Learning Rate Schedulers
  1. `StepLR` ✅
  2. `CyclicLR` ❌
  3. `OneCylceLR` ❌
  4. Try Learning Rate WarmUp ❌
- Try Label Smoothing
- Try DropBlock and DropOut see which is better ❌
- Check for Gradient Clipping ❌

### Model Callibration

- Try Model Calibration
  1. Temperature Scaling for Calibration
  2. Calbration with Brier Score
 
### Visualization and Debugging

- Grad-CAM
- TensorBoard
- Weights and Biases
- Update Ratios

# Train and Test Loaders

In [2]:
import os
import cv2
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
import torchvision.transforms as transforms
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import matplotlib.colors as mcolors
import torch
from sklearn.metrics import f1_score

In [3]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the subfolders (0, 1, ..., 9).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Iterate over each subdirectory (0, 1, ..., 9)
        for label in range(10):
            label_dir = os.path.join(root_dir, str(label))
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Calculating the Mean and Std for the dataset

In [4]:
# import torch
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader

# # Define a simple transformation to just convert images to tensors
# transform = transforms.Compose([
#     transforms.Resize((300, 300)),
#     transforms.ToTensor()
# ])

# # Load your dataset (replace with your dataset path)
# dataset = ImageDataset(root_dir="/kaggle/input/identify-the-birds/Birds/train", transform=transform)
# loader = DataLoader(dataset, batch_size=200, shuffle=False)

# # Function to calculate mean and std
# def calculate_mean_std(loader):
#     mean = 0.0
#     std = 0.0
#     total_images = 0

#     for images, _ in loader:
#         batch_samples = images.size(0)  # Batch size (number of images in the batch)
#         images = images.view(batch_samples, images.size(1), -1)  # Flatten the height and width
#         mean += images.mean(2).sum(0)
#         std += images.std(2).sum(0)
#         total_images += batch_samples

#     mean /= total_images
#     std /= total_images
#     return mean, std

# mean, std = calculate_mean_std(loader)

# print(f"Mean: {mean}")
# print(f"Std: {std}")

Size - (224, 224)
> Mean: tensor([0.4838, 0.4930, 0.4104]) \
Std: tensor([0.1881, 0.1875, 0.1886]

Size - (528, 528)
> Mean: tensor([0.4839, 0.4930, 0.4104]) \
Std: tensor([0.1924, 0.1919, 0.1931])

Size - (300, 300)
> Mean: tensor([0.4839, 0.4931, 0.4105]) \
Std: tensor([0.1898, 0.1892, 0.1904]))

# Loading Data

In [5]:
# Training Data Augmentation
train_transform = transforms.Compose([
    transforms.Resize((300, 300)),  # Resize to a consistent size
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(15),  # Randomly rotate images by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Apply color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4838, 0.4931, 0.4105], std=[0.1924, 0.1919, 0.1931])
])

# Testing/Validation Data Transform
test_transform = transforms.Compose([
    transforms.Resize((300, 300)),  # Resize to a consistent size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4838, 0.4931, 0.4105], std=[0.1924, 0.1919, 0.1931])
])

In [6]:
root_dir = "/kaggle/input/identify-the-birds/Birds"
train_dir = os.path.join(root_dir, "train")

dataset = ImageDataset(root_dir=train_dir)
labels = dataset.labels
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

## Preparing Data

In [7]:
for train_idx, test_idx in split.split(dataset.image_paths, labels):
    train_dataset = Subset(dataset, train_idx)
    train_dataset.dataset.transform = train_transform

    test_dataset = Subset(dataset, test_idx)
    test_dataset.dataset.transform = test_transform

In [8]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# EDA

In [9]:
# for idx, (images, labels) in enumerate(train_loader):
#     print(images.shape)

#     # printing the image to see
#     image_numpy = images[0].permute(1, 2, 0).numpy()
#     plt.imshow(image_numpy)
#     plt.axis('off')
#     plt.show()
    
#     print(labels.shape)
#     print(idx)
#     if (idx > 1):
#         break

### Getting a Base Line Model to run, to check what input size gives the best results

In [9]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch
import argparse
import os

In [10]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Skip connection
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return self.relu(out)

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # Initial convolution
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

            # Layer 1
            ResNetBlock(64, 64),
            ResNetBlock(64, 64),

            # Layer 2
            ResNetBlock(64, 128, stride=2),
            ResNetBlock(128, 128),

            # Layer 3
            ResNetBlock(128, 256, stride=2),
            ResNetBlock(256, 256),

            # Layer 4
            ResNetBlock(256, 512, stride=2),
            ResNetBlock(512, 512),

            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 10),  # Adjust the output layer for 8 classes
        )

    def forward(self, x):
        return self.model(x)

In [12]:
# ----------------- model training ----------------- #
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Available device: {device}")

model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
print(f"No. of model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Available device: cuda
No. of model parameters: 11186442


In [13]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

+------------------------+------------+
|        Modules         | Parameters |
+------------------------+------------+
|     model.0.weight     |    9408    |
|      model.0.bias      |     64     |
|     model.1.weight     |     64     |
|      model.1.bias      |     64     |
|  model.4.conv1.weight  |   36864    |
|   model.4.conv1.bias   |     64     |
|   model.4.bn1.weight   |     64     |
|    model.4.bn1.bias    |     64     |
|  model.4.conv2.weight  |   36864    |
|   model.4.conv2.bias   |     64     |
|   model.4.bn2.weight   |     64     |
|    model.4.bn2.bias    |     64     |
|  model.5.conv1.weight  |   36864    |
|   model.5.conv1.bias   |     64     |
|   model.5.bn1.weight   |     64     |
|    model.5.bn1.bias    |     64     |
|  model.5.conv2.weight  |   36864    |
|   model.5.conv2.bias   |     64     |
|   model.5.bn2.weight   |     64     |
|    model.5.bn2.bias    |     64     |
|  model.6.conv1.weight  |   73728    |
|   model.6.conv1.bias   |    128     |


11186442

In [ ]:
from torch.optim.lr_scheduler import StepLR

# Define optimizer and StepLR scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Decays learning rate by 0.1 every 5 epochs

# Training loop
num_epochs = 20

# Initialize logging
train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
train_f1_scores, test_f1_scores = [], []

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    y_true_train, y_pred_train = [], []  # To store true labels and predictions for F1 score

    # Training phase
    for images, labels in train_loader:
        images = images.to(device).float()  # Move images to GPU
        labels = labels.to(device).long()   # Move labels to GPU

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy calculation for training data
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Accumulate true labels and predictions for F1 score
        y_true_train.extend(labels.cpu().numpy())  # Move labels back to CPU
        y_pred_train.extend(predicted.cpu().numpy())  # Move predictions back to CPU

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # Compute F1 score for training
    f1_macro_train = f1_score(y_true_train, y_pred_train, average='macro')
    f1_micro_train = f1_score(y_true_train, y_pred_train, average='micro')
    f1_train = (f1_macro_train + f1_micro_train) / 2

    # Append training metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    train_f1_scores.append(f1_train)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    y_true_test, y_pred_test = [], []  # To store true labels and predictions for F1 score

    with torch.no_grad():  # No need to calculate gradients during validation
        for images, labels in test_loader:
            images = images.to(device).float()  # Move images to GPU
            labels = labels.to(device).long()   # Move labels to GPU

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            # Accuracy calculation for test data
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Accumulate true labels and predictions for F1 score
            y_true_test.extend(labels.cpu().numpy())
            y_pred_test.extend(predicted.cpu().numpy())

    test_loss = running_loss / len(test_loader)
    test_acc = correct / total

    # Compute F1 score for validation
    f1_macro_test = f1_score(y_true_test, y_pred_test, average='macro')
    f1_micro_test = f1_score(y_true_test, y_pred_test, average='micro')
    f1_test = (f1_macro_test + f1_micro_test) / 2

    # Append test metrics
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    test_f1_scores.append(f1_test)

    # Adjust learning rate
    scheduler.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Train F1: {f1_train:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {f1_test:.4f}')

Epoch [1/20], Train Loss: 1.4383, Train Acc: 0.4768, Train F1: 0.4719, Test Loss: 1.2977, Test Acc: 0.5572, Test F1: 0.5480
Epoch [2/20], Train Loss: 0.8675, Train Acc: 0.7139, Train F1: 0.7127, Test Loss: 1.0646, Test Acc: 0.6837, Test F1: 0.6770
Epoch [3/20], Train Loss: 0.6284, Train Acc: 0.7973, Train F1: 0.7958, Test Loss: 0.7431, Test Acc: 0.7555, Test F1: 0.7512
Epoch [4/20], Train Loss: 0.4974, Train Acc: 0.8394, Train F1: 0.8376, Test Loss: 1.0821, Test Acc: 0.6862, Test F1: 0.6804
Epoch [5/20], Train Loss: 0.4028, Train Acc: 0.8624, Train F1: 0.8605, Test Loss: 0.6944, Test Acc: 0.7826, Test F1: 0.7798
Epoch [6/20], Train Loss: 0.2414, Train Acc: 0.9286, Train F1: 0.9270, Test Loss: 0.3691, Test Acc: 0.8865, Test F1: 0.8846
Epoch [7/20], Train Loss: 0.1844, Train Acc: 0.9511, Train F1: 0.9498, Test Loss: 0.3599, Test Acc: 0.8931, Test F1: 0.8906
Epoch [8/20], Train Loss: 0.1548, Train Acc: 0.9640, Train F1: 0.9630, Test Loss: 0.3364, Test Acc: 0.8941, Test F1: 0.8919
Epoch [9

In [ ]:
# Plotting training and validation loss curves
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation accuracy curves
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Function to calculate and plot confusion matrix
def plot_confusion_matrix(loader, model, device):
    all_preds = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device).float()
            labels = labels.to(device).long()
            outputs = model(images)

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Example usage
plot_confusion_matrix(test_loader, model, device)

# GRAD-CAM stuff...

In [6]:
dataset = ImageDataset(root_dir='/kaggle/input/identify-the-birds/Birds/train', transform=test_transform)
loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
model = CNNModel().to(device)
model.load_state_dict(torch.load('/kaggle/working/bird.pth'))

/tmp/ipykernel_30/194968483.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/bird.pth'))


<All keys matched successfully>

In [13]:
import pandas as pd

y_hat = pd.read_csv('/kaggle/working/bird.csv')

In [27]:
y_hat.loc[0].values[0]

0

In [28]:
total = 0
correct = 0

y_true_labels = []
y_hat_labels = []

for image, label in loader:
    label = label.to(device).long()
    if (label.item() == y_hat.loc[total].values[0]):
        correct += 1
    y_true_labels.append(label.item())
    y_hat_labels.append(y_hat.loc[total].values[0])
    total += 1
    

In [29]:
accuracy = correct/total

In [30]:
accuracy

0.9244979919678715

In [32]:
F1 = (f1_score(y_true_labels, y_hat_labels, average='micro') + f1_score(y_true_labels, y_hat_labels, average='macro'))/2

In [33]:
F1

0.9223807125639087